# The Simple Probabilistic Model

The simple probabilistic model (SPM) is the first model used in our thesis. It is a discretization of the model presentented in Chapter 10.2 in Cartea et al.'s book _Algorithmic and High-Frequency Trading_.

It can be summarized as follows (for the full definition, see our **[report](https://github.com/KodAgge/Reinforcement-Learning-for-Market-Making/blob/main/Reinforcement%20Learning%20for%20Market%20Making.pdf)**):
> * The time _t_ can take integer values between _0_ and _T_.
>
> * The midprice _S<sub>t</sub>_ is a Brownian motion rounded to the closest tick.
>
> * The market maker has to quote bid and ask prices every second.
>
> * The market maker can put the bid and ask depths at _d_ different levels, from _0_ to _d - 1_ ticks away from the mid price.
>
> * The cash process _X<sub>t</sub>_ denotes the market maker's cash at time _t_.
>
> * The inventory process _Q<sub>t</sub>_ denotes the market maker's inventory at time _t_.
>
> * The value process _V<sub>t</sub>_ denotes the value of the market maker's position at time _t_, that is its cash plus the value of its current inventory.
>
> * The market maker can see the current time and its inventory _(t,Q<sub>t</sub>)_ before taking an action.
>
> * At time _t = T_ the market maker is forced to liquidate its position.

The _tick_ is the smallest tradeable unit of the underlying, for instance $0.01 of AAPL.

Based on Cartea et al.'s definition, an analytically optimal strategy can be defined, with which we want to compare strategies derived with Q-learning. However, there is no guarantee that these strategies are optimal in the discretized version of the model.

An example of the optimal bid depths for a specific set of model parameters is shown in the figure below. **Note** that these depths are _not_ discretized in terms of depth, that is they're not rounded to the closest tick.

<div>
    <img src="images/ContinuousBid30.png"/>
</div>


## The Q-learning

After that short introduction, it's time for some reinforcement learning in the form of Q-learning.

We start by importing the needed file.

In [1]:
# import the Q-learning file for the simple probabilistic model
from simple_model_evaluation import *

Now we have to decide on the parameters we want to use for the environment and the hyperparameters we want to use for the Q-learning.

For the model we choose an episode length of *T = 20* and a running inventory penalty of *$\phi$ = 10<sup>-4</sup>*.

In [2]:
model_params = {
                "d": 4,         # the number of different quotation depths that can be chosen from
                "T": 20,        # the length of the episode
                "dp": 0.01,     # the tick size
                "min_dp": 0,    # the minimum number of ticks from the mid price that is allowed to put prices at
                "phi": 1e-4     # the running inventory penalty
}

We now have to decide which hyperparameter values we want to use.

There is not much to choose here. We have to decide on the parameter schemes for the epislon-greedy policy, the learning rate and the exploring starts. Finally we need to decide how long we want to train for, how many times we want to train and how long we want to evalute for.

> *\_start* indicates the starting value of the parameter.
>
> *\_end* indicates the final value of the parameter.
>
> *\_cutoff* indicates when the final value is reached, i.e. 0.5 means after 50% of the training.

In [3]:
Q_learning_params = {
        # epsilon-greedy values (linear decay)
        "epsilon_start": 1,
        "epsilon_end": 0.05,
        "epsilon_cutoff": 0.5,

        # learning-rate values (exponential decay)
        "alpha_start": 0.5,
        "alpha_end": 0.001,
        "alpha_cutoff": None,

        # exploring starts values (linear decay)
        "beta_start": 1,
        "beta_end": 0.05,
        "beta_cutoff": 0.5,
        "exploring_starts": True
}

hyperparams = {
        "n_train" : 1e5,
        "n_test" : 1e4,
        "n_runs" : 4
}

Finally we decide where to save our results.

In [4]:
# naming the folder where the results will be saved
folder_mode = True
folder_name = "spm_example"
save_mode = True

We're now ready for the Q-learning!

This is easily done with the function *Q\_learning\_comparison*.

In [5]:
Q_learning_comparison(
    **hyperparams,
    args                = model_params,
    Q_learning_args     = Q_learning_params,
    folder_mode         = folder_mode,
    folder_name         = folder_name,
    save_mode           = save_mode
)

RUN 1 IN PROGRESS...
	Episode 20000 (20%), 0:02:15.900000 remaining of this run
	Episode 40000 (40%), 0:01:31.910000 remaining of this run
	Episode 60000 (60%), 0:00:55.660000 remaining of this run
	Episode 80000 (80%), 0:00:26.240000 remaining of this run
	Episode 100000 (100%), 0:00:00 remaining of this run
THE FOLDER spm_example ALREADY EXISTS
...FINISHED IN 0:02:06.430000
0:06:19.300000 REMAINING OF THE TRAINING
RUN 2 IN PROGRESS...
	Episode 20000 (20%), 0:02:24.510000 remaining of this run
	Episode 40000 (40%), 0:01:40.060000 remaining of this run
	Episode 60000 (60%), 0:01:01.650000 remaining of this run
	Episode 80000 (80%), 0:00:29.790000 remaining of this run
	Episode 100000 (100%), 0:00:00 remaining of this run
THE FOLDER spm_example ALREADY EXISTS
...FINISHED IN 0:02:26.520000
0:04:53.050000 REMAINING OF THE TRAINING
RUN 3 IN PROGRESS...
	Episode 20000 (20%), 0:02:18.700000 remaining of this run
	Episode 40000 (40%), 0:01:35.560000 remaining of this run
	Episode 60000 (60%),

## Evaluating the strategies

We can now have a look at the images that were saved when running *Q\_learning\_comparison*.

Let's first have a look at the reward and the state-value at (0,0) during training.

<div>
    <img src="results/simple_model/spm_example/results_graph.png"/>
</div>

In this image it looks like that the Q-learning has converged, however, it has not. It has to be trained for longer, which will be evident in the coming images.

We can also have a look the learnt strategies. The figure below shows the learnt bid depths.

<div>
    <img src="results/simple_model/spm_example/opt_bid_strategy.png" width="500"/>
</div>

Furthermore, we can compare the average rewards of the Q-learning strategies versus benchmarking strategies. These are displayed in the boxplot below.

<div>
    <img src="results/simple_model/spm_example/box_plot_benchmarking.png"/>
</div>

We can also view these results in table form.


In [1]:
f = open("results/simple_model/spm_example/table_benchmarking")
print(f.read())
f.close()

strategy                 mean reward    std reward
---------------------  -------------  ------------
analytical_discrete        0.127752      0.0764701
analytical_continuous      0.132987      0.0717148
constant (d=2)             0.0987989     0.0796333
random                     0.0618333     0.0958115
Q_learning (best run)      0.123278      0.0772611
Q_learning (average)       0.127178      0.0785348


All in all it looks like the Q-learning has been able to find decent strategies, but it needs to train for longer in order to find strategies that equal the analytical strategies in performance.

## More results?

There are a lot more figures and tables to explore which can be found in the **[spm_example](https://github.com/KodAgge/Reinforcement-Learning-for-Market-Making/tree/main/code/results/simple_model/spm_example)** folder.